In [1]:
from activity_reader import *
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import numpy as np

In [2]:
def calculate_gradient(distance, elevation):
    gradient = []
    for i in range(len(distance)-1):
        delta_elevation = elevation[i] - elevation[i+1]
        delta_distance = distance[i] - distance[i+1]
        if delta_distance != 0:
            gradient.append(delta_elevation/delta_distance)
        else:
            gradient.append(0)
    gradient.append(0)
    return gradient

In [3]:
# Simplified track to optimize over
distance = np.arange(0,100)
slope = np.array([0]*100)

In [4]:
import numpy as np
import control as ct 
import control.optimal as obc
import matplotlib.pyplot as plt


def bicycle_update(t, x, u, params={}):
    """Bicycle dynamics for control system.
    
    Parameters
    ----------
    x : array
            System state: [position, velocity, remaining anaerobic capacity]
    u : array
            System input: [power]

    Returns
    ----------
    array:
        [velocity, acceleration, change in anaerobic capacity] 
    """
    

    # System parameters
    m = params.get('m', 85)
    g = params.get('g', 9.81)
    my = params.get('my', 0.004)
    b0 = params.get('b0', 0.091)
    b1 = params.get('b1', 0.0087)
    Iw = params.get('Iw', 0.14)
    r = params.get('r', 0.33)
    Cd = params.get('Cd', 0.7)
    rho = params.get('rho', 1.2)
    A = params.get('A', 0.4)
    eta = params.get('eta', 1)
    slope = params.get('slope', np.array([0]*1000))
    w_prime = params.get('w_prime', 26630)
    cp = params.get('cp', 265)

    # Variables for states and input
    v = x[1]
    w_bal = x[2]
    power = u[0]
    dw_bal = 0
    if power < cp:
        dw_bal = (1-w_bal/w_prime)*(cp-power)
    else:
        dw_bal = -(power - cp)


    dv = 1/v * 1/(m + Iw/r**2) * (eta*power - m*g*v*slope[int(t)] - my*m*g*v - b0*v - b1*v**2 - 0.5*Cd*rho*A*v**3)
    
    return np.array([v, dv, dw_bal])

def bicycle_output(t, x, u, params):
    return x

bicycle_system = ct.NonlinearIOSystem(bicycle_update, bicycle_output, states=3, name='bicycle', inputs=('u'), outputs=('p', 'v', 'w'))

In [5]:
# Solving the optimal control problem:
def cost_function(t, x, u, params={}):
    return t

mass_rider = 78
mass_bike = 8
params = {
    'm': mass_bike + mass_rider,
    'slope': slope,
    'g': 9.81,
    'my': 0.004,
    'b0': 0.091,
    'b1': 0.0087,
    'Iw': 0.14,
    'rw': 0.33,
    'Cd': 0.7,
    'rho': 1.2,
    'A': 0.4,
    'eta': 1
}

x0 = [0, 0.1, 26630]
t = np.linspace(0, 1000, 1000, endpoint=True)
input_constraint = obc.input_range_constraint(bicycle_system, [0], [600])
output_constraint = obc.output_range_constraint(bicycle_system, [0, 0.01, 1], [distance[-1], 50, 26630])
initial_guess = np.full((bicycle_system.ninputs, len(t)), 265)
ivp_kwargs = {'args': {'params': params}}
minimize_options = {'tolerance': 0.01}


ocp = obc.OptimalControlProblem(bicycle_system, t, integral_cost=cost_function, initial_guess=initial_guess, minimize_options=minimize_options)

C:\Users\brokk\AppData\Local\Temp\ipykernel_4232\3132609527.py:51: RuntimeWarning: divide by zero encountered in scalar divide
  dv = 1/v * 1/(m + Iw/r**2) * (eta*power - m*g*v*slope[int(t)] - my*m*g*v - b0*v - b1*v**2 - 0.5*Cd*rho*A*v**3)
C:\Users\brokk\AppData\Local\Temp\ipykernel_4232\3132609527.py:51: RuntimeWarning: invalid value encountered in scalar multiply
  dv = 1/v * 1/(m + Iw/r**2) * (eta*power - m*g*v*slope[int(t)] - my*m*g*v - b0*v - b1*v**2 - 0.5*Cd*rho*A*v**3)


RuntimeError: solve_ivp failed: Required step size is less than spacing between numbers.

In [ ]:
def cost_function(u, x):
    return -x[1]

x0 = [0, 0.1, 26630]
t = np.linspace(0, 100, 100, endpoint=True)
input_constraint = obc.input_range_constraint(bicycle_system, [1], [600])
output_constraint = obc.output_range_constraint(bicycle_system, [0, 1, 1], [distance[-1], 50, 26630])
initial_guess = np.full((bicycle_system.ninputs, len(t)), 265)

ocp = obc.solve_ocp(bicycle_system, t, x0, cost_function, solve_ivp_method='RK45', initial_guess=initial_guess, print_summary=True)

C:\Users\brokk\AppData\Local\Temp\ipykernel_6072\3132609527.py:51: RuntimeWarning: divide by zero encountered in scalar divide
  dv = 1/v * 1/(m + Iw/r**2) * (eta*power - m*g*v*slope[int(t)] - my*m*g*v - b0*v - b1*v**2 - 0.5*Cd*rho*A*v**3)
C:\Users\brokk\AppData\Local\Temp\ipykernel_6072\3132609527.py:51: RuntimeWarning: invalid value encountered in scalar multiply
  dv = 1/v * 1/(m + Iw/r**2) * (eta*power - m*g*v*slope[int(t)] - my*m*g*v - b0*v - b1*v**2 - 0.5*Cd*rho*A*v**3)


RuntimeError: solve_ivp failed: Required step size is less than spacing between numbers.